In [1]:
from osgeo import gdal, gdalconst, ogr, osr
import numpy as np
import rasterio
import glob, os

In [4]:
num_channel = 8
dir_img = r"/home/skm/SKM16/Work/Npark_planet/Z_Tat_ca_anh_roi_rac/img_origin_remove_cloud/t3/mosaic"

In [ ]:

# img_for_follow = r"/home/skm/SKM16/Work/Npark_planet/image_roi_rac/20220330_023353_67_2460_3B_AnalyticMS_SR_8b_clip.tif"
img_for_follow = r"/home/skm/SKM16/Work/Npark_planet/Z_Tat_ca_anh_roi_rac/img_origin_remove_cloud/t3/20220330_023353_67_2460_3B_AnalyticMS_SR_8b_clip.tif"
dataset = gdal.Open(img_for_follow, gdal.GA_ReadOnly)
dict_band_min_max_percentile_follow = {k: dict(max=0, min=0) for k in range(num_channel)}

for i_chan in range(num_channel):
    band = dataset.GetRasterBand(i_chan+1).ReadAsArray().astype(np.float16)
    band[band==0] = np.nan
    dict_band_min_max_percentile_follow[i_chan]['max'] = np.nanpercentile(band, 98)
    dict_band_min_max_percentile_follow[i_chan]['min'] = np.nanpercentile(band, 2)

In [3]:
list_fp = glob.glob(os.path.join(dir_img, "*.tif"))
for image_path in list_fp:
    dataset_ori = gdal.Open(image_path, gdal.GA_ReadOnly)
    dict_band_min_max_percentile_ori = {k: dict(max=0, min=0) for k in range(num_channel)}

    for i_chan in range(num_channel):
        band = dataset_ori.GetRasterBand(i_chan+1).ReadAsArray().astype(np.float16)
        band[band==0] = np.nan
        dict_band_min_max_percentile_ori[i_chan]['max'] = np.nanpercentile(band, 98)
        dict_band_min_max_percentile_ori[i_chan]['min'] = np.nanpercentile(band, 2)


        src = rasterio.open(image_path)
        meta = src.meta
        img_ = src.read()
        img_percentile = np.empty(img_.shape)
        for i in range(8):
            idxnodata = np.where(img_[i]==meta['nodata'])
            img_percentile[i] = np.interp(img_[i], (dict_band_min_max_percentile_ori[i]['min'], dict_band_min_max_percentile_ori[i]['max']), 
                                                (dict_band_min_max_percentile_follow[i]['min'], dict_band_min_max_percentile_follow[i]['max']))
            img_percentile[i][idxnodata]=meta['nodata']
            img_percentile[i].astype('uint16')



SyntaxError: unexpected EOF while parsing (<ipython-input-3-427845fc1b22>, line 2)